In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/users/tma2/miniconda3/envs/llamafactory/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model_name_or_path = "/users/tma2/Projects/Halluc/models/qwen3-4b-instruct-2507_gsm8k_bt_sft_lora"

In [10]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.15s/it]


In [11]:
question = "What does <|BACKTRACK|> means?"

In [12]:
def format_question(question: str) -> str:
    messages = [
        {"role": "user", "content": question}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return text

In [16]:
formated_input = format_question(question)
print("Formatted input:", formated_input)

Formatted input: <|im_start|>user
What does <|BACKTRACK|> means?<|im_end|>
<|im_start|>assistant



In [24]:
model_inputs = tokenizer([formated_input], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 


In [25]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=False).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content) # no opening <think> tag
print("content:", content)

thinking content: 
content: The symbol **℮** is known as the **"trademark sign"** or **"trademark symbol"**. It is used to indicate that a particular name, logo, or product is a registered trademark. This symbol is often used in branding and intellectual property to show that the associated product or service is legally protected under trademark law.

For example, you might see it before a company name like ™Nike or ™Apple to indicate that the brand is trademarked.

**Note:** The symbol ℥ (which looks like a small "e" with a line through it) is sometimes confused with ™, but they are different:
- **™** = Trademark (used for unregistered trademarks)
- **®** = Registered trademark (used for registered trademarks)

So, if you saw "℮", it likely meant "trademark" in a specific context, though it is not as commonly used as ™ or ®.


In [26]:
model_inputs["input_ids"]

tensor([[151644,    872,    198,   3838,   1558,    220, 151669,   3363,     30,
         151645,    198, 151644,  77091,    198]], device='cuda:0')

In [29]:
tokenizer(["<|BACKTRACK|>"])

{'input_ids': [[151669]], 'attention_mask': [[1]]}